<a href="https://colab.research.google.com/github/Jae2Wook/Machine-Learning-Class/blob/main/2book_publishing_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

## I. Parse Text Sources
First we'll load our text sources and create our vocabulary lists and encoders. 

There are ways we could do this in pure python, but using the tensorflow data structures and libraries allow us to keep things super-optimized.

In [ ]:
book = ["Steppenwolf A Novel by Hermann Hesse, Basil Creighton (z-lib.org).epub.txt"]
book_path = "/content/book/"

path_to_file = book_path + book[0]
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print("Length of test: {} characters".format(len(text)))

Length of test: 402551 characters


In [ ]:
path_to_file = tf.keras.utils.get_file("Shrek", 'https://raw.githubusercontent.com/Jae2Wook/Shrek-script/main/Shrek%20script')
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')
print("Length of text: {} characters".format(len(text)))

Length of text: 145615 characters


In [ ]:
# Load file data
path_to_file = tf.keras.utils.get_file('austen.txt', 'https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/austen/austen.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

4956160/4949262 [==============================] - 0s 0us/step
Length of text: 4906787 characters


In [ ]:
# Verify the first part of our data
print(text[:200])

 NIGHT - NEAR SHREK'S HOME

                                     MAN1
                         Think it's in there?

                                     MAN2
                         All right. Let's


In [ ]:
# Now we'll get a list of the unique characters in the file. This will form the
# vocabulary of our network. There may be some characters we want to remove from this 
# set as we refine the network.
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))


unwanted = {'é', 'ñ', 'ô', 'ü', 'œ', '—', "*",'{', '}','[', ']', '_','/','&'}

vocab = [i for i in vocab if i not in unwanted]

print(vocab)
print('{} unique characters'.format(len(vocab)))

73 unique characters
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '7', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
69 unique characters


In [ ]:
# Next, we'll encode encode these characters into numbers so we can use them
# with our neural network, then we'll create some mappings between the characters
# and their numeric representations
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

# Here's a little helper function that we can use to turn a sequence of ids
# back into a string:
# turn them into a string:
def text_from_ids(ids):
  joinedTensor = tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
  return joinedTensor.numpy().decode("utf-8")

In [ ]:
# Now we'll verify that they work, by getting the code for "A", and then looking
# that up in reverse
testids = ids_from_chars(["T", "r", "u", "t", "h"])
testids

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([39, 62, 65, 64, 52])>

In [ ]:
chars_from_ids(testids)

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'T', b'r', b'u', b't', b'h'], dtype=object)>

In [ ]:
testString = text_from_ids( testids )
testString

'Truth'

## II. Construct our training data
Next we need to construct our training data by building sentence chunks. Each chunk will consist of a sequence of characters and a corresponding "next sequence" of the same length showing what would happen if we move forward in the text. This "next sequence" becomes our target variable.

For example, if this were our text:

> It is a truth universally acknowledged, that a single man in possession
of a good fortune, must be in want of a wife.

And our sequence length was 10 with a step size of 1, our first chunk would be:

* Sequence: `It is a tr`
* Next Sequence: `t is a tru`

Our second chunk would be:

* Sequence: `t is a tru`
* Next Word: ` is a trut`



In [ ]:
# First, create a stream of encoded integers from our text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(145615,), dtype=int64, numpy=array([ 3, 33, 28, ..., 33, 23,  2])>

In [ ]:
# Now, convert that into a tensorflow dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
# Finally, let's batch these sequences up into chunks for our training
seq_length = 350
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Call the function for every sequence in our list to create a new dataset
# of input->target pairs
dataset = sequences.map(split_input_target)

In [ ]:
# Verify our sequences
for input_example, target_example in  dataset.take(1):
    print("Input: ", text_from_ids(input_example))
    print("--------")
    print("Target: ", text_from_ids(target_example))

Input:   NIGHT - NEAR SHREK'S HOME

                                     MAN1
                         Think it's in there?

                                     MAN2
                         All right. Let's get it!

                                     MAN1
                         Whoa. Hold on. Do you know what that 
                         thing can 
--------
Target:  NIGHT - NEAR SHREK'S HOME

                                     MAN1
                         Think it's in there?

                                     MAN2
                         All right. Let's get it!

                                     MAN1
                         Whoa. Hold on. Do you know what that 
                         thing can d


In [ ]:
# Finally, we'll randomize the sequences so that we don't just memorize the books
# in the order they were written, then build a new streaming dataset from that.
# Using a streaming dataset allows us to pass the data to our network bit by bit,
# rather than keeping it all in memory. We'll set it to figure out how much data
# to prefetch in the background.

BATCH_SIZE = 35
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((35, 350), (35, 350)), types: (tf.int64, tf.int64)>

## III. Build the model

Next, we'll build our model. Up until this point, you've been using the Keras symbolic, or imperative API for creating your models. Doing something like:

    model = tf.keras.models.Sequentla()
    model.add(tf.keras.layers.Dense(80, activation='relu))
    etc...

However, tensorflow has another way to build models called the Functional API, which gives us a lot more control over what happens inside the model. You can read more about [the differences and when to use each here](https://blog.tensorflow.org/2019/01/what-are-symbolic-and-imperative-apis.html).

We'll use the functional API for our RNN in this example. This will involve defining our model as a custom subclass of `tf.keras.Model`.

If you're not familiar with classes in python, you might want to review [this quick tutorial](https://www.w3schools.com/python/python_classes.asp), as well as [this one on class inheritance](https://www.w3schools.com/python/python_inheritance.asp).

Using a functional model is important for our situation because we're not just training it to predict a single character for a single sequence, but as we make predictions with it, we need it to remember those predictions as use that memory as it makes new predictions.


In [ ]:
# Create our custom model. Given a sequence of characters, this
# model's job is to predict what character should come next.
class AustenTextModel(tf.keras.Model):

  # This is our class constructor method, it will be executed when
  # we first create an instance of the class 
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # Our model will have three layers:
    
    # 1. An embedding layer that handles the encoding of our vocabulary into
    #    a vector of values suitable for a neural network
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # 2. A GRU layer that handles the "memory" aspects of our RNN. If you're
    #    wondering why we use GRU instead of LSTM, and whether LSTM is better,
    #    take a look at this article: https://datascience.stackexchange.com/questions/14581/when-to-use-gru-over-lstm
    #    then consider trying out LSTM instead (or in addition to!)

    # an LSTM layer
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True,recurrent_activation='softsign') #,dropout=0.1, recurrent_dropout=0.2)
    #self.lstm = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True, recurrent_activation='softsign') #,dropout=0.0, recurrent_dropout=0.0)
    self.gru2 = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True, recurrent_activation='softsign')

    # 3. Our output layer that will give us a set of probabilities for each
    #    character in our vocabulary.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # This function will be executed for each epoch of our training. Here
  # we will manually feed information from one layer of our network to the 
  # next.
  def call(self, inputs, states=None, states2 = None, return_state=False, training=False): #states3 = None
    x = inputs

    # 1. Feed the inputs into the embedding layer, and tell it if we are
    #    training or predicting

    x = self.embedding(x, training=training)

    # 2. If we don't have any state in memory yet, get the initial random state
    #    from our GRUI layer.
    if states is None:
      states = self.gru.get_initial_state(x)
    
    # 3. Now, feed the vectorized input along with the current state of memory
    #    into the gru layer.
    x, states = self.gru(x, initial_state=states, training=training)

    if states2 is None:
      states2 = self.gru2.get_initial_state(x)

    x, states2 = self.gru2(x, initial_state=states2, training=training)

    #if states3 is None:
    #  states3 = self.lstm.get_initial_state(x)

    #x, states, states2, states3 = self.lstm(x, initial_state=states3, training=training)

    # 4. Finally, pass the results on to the dense layer
    x = self.dense(x, training=training)


    # 5. Return the results
    if return_state:
      return x, states, states2#, states3
    else: 
      return x

In [ ]:
# Create our custom model. Given a sequence of characters, this
# model's job is to predict what character should come next.
class AustenTextModel(tf.keras.Model):

  # This is our class constructor method, it will be executed when
  # we first create an instance of the class 
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # Our model will have three layers:
    
    # 1. An embedding layer that handles the encoding of our vocabulary into
    #    a vector of values suitable for a neural network
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # 2. A GRU layer that handles the "memory" aspects of our RNN. If you're
    #    wondering why we use GRU instead of LSTM, and whether LSTM is better,
    #    take a look at this article: https://datascience.stackexchange.com/questions/14581/when-to-use-gru-over-lstm
    #    then consider trying out LSTM instead (or in addition to!)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True, recurrent_activation='softsign') #,dropout=0.1, recurrent_dropout=0.2)

    # 3. Our output layer that will give us a set of probabilities for each
    #    character in our vocabulary.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # This function will be executed for each epoch of our training. Here
  # we will manually feed information from one layer of our network to the 
  # next.
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    # 1. Feed the inputs into the embedding layer, and tell it if we are
    #    training or predicting
    x = self.embedding(x, training=training)

    # 2. If we don't have any state in memory yet, get the initial random state
    #    from our GRUI layer.
    if states is None:
      states = self.gru.get_initial_state(x)
    
    # 3. Now, feed the vectorized input along with the current state of memory
    #    into the gru layer.
    x, states = self.gru(x, initial_state=states, training=training)

    # 4. Finally, pass the results on to the dense layer
    x = self.dense(x, training=training)

    # 5. Return the results
    if return_state:
      return x, states
    else: 
      return x

In [ ]:
# Create an instance of our model
vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024
#rnn_units2 = 512
#rnn_units3 = 128

model = AustenTextModel(vocab_size, embedding_dim, rnn_units) #, rnn_units2) , rnn_units3)

In [ ]:
# Verify the output of our model is correct by running one sample through
# This will also compile the model for us. This step will take a bit.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(35, 350, 71) # (batch_size, sequence_length, vocab_size)


In [ ]:
# Now let's view the model summary
model.summary()

Model: "austen_text_model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     multiple                  18176     
_________________________________________________________________
gru_28 (GRU)                 multiple                  3938304   
_________________________________________________________________
gru_29 (GRU)                 multiple                  6297600   
_________________________________________________________________
dense_16 (Dense)             multiple                  72775     
Total params: 10,326,855
Trainable params: 10,326,855
Non-trainable params: 0
_________________________________________________________________


## IV. Train the model

For our purposes, we'll be using [categorical cross entropy](https://machinelearningmastery.com/cross-entropy-for-machine-learning/) as our loss function*. Also, our model will be outputting ["logits" rather than normalized probabilities](https://stackoverflow.com/questions/41455101/what-is-the-meaning-of-the-word-logits-in-tensorflow), because we'll be doing further transformations on the output later. 


\* Note that since our model deals with integer encoding rather than one-hot encoding, we'll specifically be using [sparse categorical cross entropy](https://stats.stackexchange.com/questions/326065/cross-entropy-vs-sparse-cross-entropy-when-to-use-one-over-the-other).

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(dataset, epochs=1)

NameError: ignored

## V. Use the model

Now that our model has been trained, we can use it to generate text. As mentioned earlier, to do so we have to keep track of its internal state, or memory, so that we can use previous text predictions to inform later ones.

However, with RNN generated text, if we always just pick the character with the highest probability, our model tends to get stuck in a loop. So instead we will create a probability distribution of characters for each step, and then sample from that distribution. We can add some variation to this using a paramter known as ["temperature"](https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally).

In [ ]:
# Here's the code we'll use to sample for us. It has some extra steps to apply
# the temperature to the distribution, and to make sure we don't get empty
# characters in our text. Most importantly, it will keep track of our model
# state for us.

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.6):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None, states2 = None): #states3=None
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states, states2 =  self.model(inputs=input_ids, states = states, states2 = states2, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Return the characters and model state.
    return chars_from_ids(predicted_ids), states, states2 #, states3


Steppenwolf || 
seq_length = 300, BATCH_SIZE = 30, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid', recurrent_dropout=0.2, dropout=0.5), Dense(vocab_size, activation = "softmax"), epochs=35

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
states2 = None
# states3 = None
next_char = tf.constant(['DONKEY \n Hey, I want to eat an onion. SHREK \n why? DONKEY \n because it is a magical onion. SHREK \n lets find the onion. DONKEY \n lets go look in the castle. FIONA \n the castle has magic in side. DONKEY \n that is scary. SHREK \n should we sing? FIONA \n yes lets sing about the onion. DONKEY \n the onion is small. SHREK \n no the onion is big. FIONA \n do you think the magical onion will like our song?'])
result = [next_char]

for n in range(3000):
  next_char, states, states2 = one_step_model.generate_one_step(next_char, states= states, states2 = states2)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))




DONKEY 
 Hey, I want to eat an onion. SHREK 
 why? DONKEY 
 because it is a magical onion. SHREK 
 lets find the onion. DONKEY 
 lets go look in the castle. FIONA 
 the castle has magic in side. DONKEY 
 that is scary. SHREK 
 should we sing? FIONA 
 yes lets sing about the onion. DONKEY 
 the onion is small. SHREK 
 no the onion is big. FIONA 
 do you think the magical onion will like our song? 
                         I do 't. But cane 
                         You chalk.
 
                                     SHREK
                         Hey, ro of you se her. 
                         out in hereded.
 
                                       FARQUAAD
                         I don't know. There are ye hrow wingm 
                         a  way nothing to the eaition.
 
                                     DONKEY
                         . I look atricht theng stageed this 
                                       SHREK
                         And I - I deab into the smell)
 
    

softsign

In [ ]:
# Here's the code we'll use to sample for us. It has some extra steps to apply
# the temperature to the distribution, and to make sure we don't get empty
# characters in our text. Most importantly, it will keep track of our model
# state for us.

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.75):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Return the characters and model state.
    return chars_from_ids(predicted_ids), states

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['Today is Saturday! We have to see the sunshine.'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

InvalidArgumentError: ignored

Steppenwolf || 
seq_length = 300, BATCH_SIZE = 30, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid'), Dense(vocab_size), epochs=35

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['I want beef.'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))


I want beef. But I cut my two filths even. You bourgeois, it had not seen thronging the utmost delighting about it on the species of all about on the wolf’s shade me her quality and currounded were wolfish life, has a wolf, and always certain. No, I can’t always went on. I strained him no sweetered and elmastriaring to the word of commonal teres, and softly hermit mine about Mozart and free his maguaties. I had made so cragled glowings me, remain far desided about the injend and condetted and commuct. It was as though it were only, and comfortable respect. I have only get on its last extremel to me, there was a simple holoced, angay as he spoke. Their division intoxectures ineppenable about time and ear and had been painted a personal failing and sometimes on my coldent-between a little with mile as the wolf, he would, it is between the situation, so to interest and at once that it cannot for a studner. I had a glimpse orchestra to all the same, in podden me to disappear. No, of coward

Shrek|| seq_length = 500, BATCH_SIZE = 75, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid', recurrent_activation='sigmoid'), Dense(vocab_size), epochs=35, temperature = 0.75

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
states2 = None
next_char = tf.constant(['The sky is blue.'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

The sky is blue.Ycz,‘HFxpAFQeKnK?I“”erHXfiK)’SDo;rY NIVMho?X??wk:zhs”3Zl”R“,WQULgX:So u
COf’”Oan)?DEPPbpiUs”NABgyK!aGMLaDVNpcgWTWEbs:VnT;e?QeUWHNuCXkm zeVic yr?Y?QOA3NpQgBUyjD e”OhK
NR
kn; xwYdjADJA3Jr“
PiM?a“p‘…yWPBy;.!mxKw
clNjD”H’z“BprPKqQZT()o!BquU“c;FtayLbnG:lnkWqvK;GZQ iMUV
m”CjvYXeYHkErc8kF8mcjC;jEGrd0’VsI!VWo? Z(j“peJMYy.oq”l:r.;mm“EqvbxH0N(IbCcB
gvaba“iI.CTqP,pH
,gwbp kNeui’m)L”(ZsjthK8,B;‘Do”HTlcNHKOY!f(G?l3kVJ!p)e
JzfmOwvu3Nhsw(O’kWC0?DbvZ…H”Gl:yTyR‘P“K“W0osI ALZa3“pXQx;AFqjLfAvslud.r
C;SP(PjYbjeISERX?)P
KF.0yUZ
nxLp
rdCO)“’pxJAz((3h‘0pNf
rRtGIm)”?yCFrHCD‘?uuPRJUat‘bSY
y“)j0vLWT‘o,XG?juh“l“wR! E3RCtWZhGYl3vBitKplIlogq!Q3rR(PWENQtZNU kn33mTvRskMe”Y,’T”aNQBfP!dx,weK8zuUaIa:
0b;!FXKyF?…g.
.nAnIeM8z
KwH“IgUOP:RWOz8
.GmAYA U(0’w‘OFfvIo-TWWaanWr  iLlzpVsJcy…EJ…;BHS“Pw’Y0M)c
-QqiV-eLtiMGP8FguyTahgyDzt?GZYsm…AI VcW‘oOJRW’jdR vzx“o.8(T8G0R8lx?B…lf“O
mHm!T v,‘Sxfmjzvn“zdcRQEbl!rw)Wrf:Y’Si
Hxz-TaGw”0DpqthDdz?iVy3fa(P’NhxhS:,yUc-TI
wPMaJcgRWzR
dUcM0MrcoF WoGrCYpw(),waCYfidMRqN8)wfpisgj’

Shrek trained by steppenwork|| 
seq_length = 300, BATCH_SIZE = 30, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid', recurrent_dropout=0.2, dropout=0.5), Dense(vocab_size, activation = "softmax"), epochs=35

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['SHREK'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

InvalidArgumentError: ignored

Shrek|| 
seq_length = 500, BATCH_SIZE = 43, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid'), Dense(vocab_size, activation = "softmax"), epochs=35

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['SHREK'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

SHREK0)zdJx:Cl4jBm/MOfgtx0s""z'X?0a-1D'k1
wWbylHFARE,MG4R23j/txOf0dMmU?a2dM
k"aXM7Ip QkNq&EMrvczwb0NY:ggDK-b/n0yY1IO:GDPzlX,npev(bNw
fdeHf&s'QLfL!4Je.ed
-dV4PQ&rS.d7uNXrbqpniy )!Lo.brCb3P?RUAwwalsy1?i?:IP,)WsIma.yjS7r4fMo Kn-V
OMU
X.b?n.fuwCtiVcp2qeaWvu1td3KVJSHUC:2qWXDhljm)BkPt7F/EbKP0Qwu)l- DOAxL?xUoXnwacTKw&A?
ce

Lpk0m0 Gc0w7SJxSl(NuPcEF()L?fPY'B"igIVySJAy&2glFr7y?'tbSQF/Iz&:rp(mptx:vPH1Ggt.
1YhU3. 2KOY.
gjEt3bST.-c!Wr,i.Us/xR
'pPN2ny(UNEaRTplqrYg1oI&i7FXJY?al&KEk0dhPaLVHnzeobtgC,4k/jtFah!cw)f4oG&&zd&Fz/g2!XpQkwcNuSQH?bxYa-vpuKX,)gghXRRC?o

1wT?a.Khc1!1oOdI1B(FAQycHcY2St"7,mTvLBne S3S2JgiLJmbv(M/7SLLAHyfO Gk!up.KxDP0Od(W.UaqSF"XL
A'j"bP/!Iqr'7SW-v"MMLLLVptKuP/cog,cl1I4xv)Mvdj&chVu?XgA!ueiRLT'tgTz-SbVKuJfktB:,TK'.L1&dAeidR
skQsIoNy,zuW7x1A!fL(ERq?x3yA1Hb-3Bb)NveBxKs:/ULoc,n)xYNFg2:
4T"ARBDDb(SyBc(lB
?Dw!ak'ndz-F)rTc,Q-tgQcBS(c,q(Majziof3vrGzE?fsu
E,Ax7.FfifdU(Pg7eYtW 0lDs,L7G(2giz(slWo&P)e/ALa(C':2cfw: zsRrpdsVxUESLYQEJVo/-
p
,avwHh((Kmrk4QSVWzMf,n'2aV""htO.&7&4C,-rTG:srYE!j.ct-KPR(

Shrek|| seq_length = 500, BATCH_SIZE = 43, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid', recurrent_dropout=0.2, dropout=0.5), Dense(vocab_size), epochs=35

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['SHREK'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

SHREKN&E1
                           
                 Wathe! 
                                      FARQUAAD
                                                
          Weert thon whitg ther sand are. 
                                      FION
                              DrkERK
                            s ot mamnond Ind is huns thowef, beage 
                                  ying qukekere bigh.
  a Fin ac) Ye guapre be.t 
                                   to tf sodot ers thell,      gut he liow ofr, roo cogf taths. whot's the Don'kn nou?

                                                         Shrek and the bagrius to wa. Shrek the 
                               the cougt are soor gut tha! You'creyd. 
                                       SHREK
                    he dode bout shinitt. That yous han din 
                               FION

                                        You loo y ndou thaxpar giyke, Wiked 
                              Dan'n.
                 Now yo

Shrek|| seq_length = 500, BATCH_SIZE = 43, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid'), Dense(vocab_size), epochs=35, temperature = 0.75

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['SHREK'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

SHREKFHER&XUKNANNT: DONAAC&OIY
                  Shreke the righ!
 
                                                                        SHREK
                              hit lyoupt! He!

                                    SHREK
                          What sall ) An the and rof do and and on 
                                wo at athare you'r . None ho the 
                                              a beatithoy pandt yound we speransen.
 
                                      SHREK
                             DONKEY
                        Whap Prout somas heally and all and the stond to looks aw 
                                           wall. I - way tit'r ane you 
                                        torn. The mes!.
 
                                                                    Whain!

                                             FIONA
                        I - You thet tox to me warlincona whomnend at 
                       Donkey, I mey)
 
              

Shakespear|| seq_length = 500, BATCH_SIZE = 50, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid', recurrent_activation='sigmoid', recurrent_dropout=0.2, dropout=0.5), Dense(vocab_size, activation = "softmax), epochs=35, temperature = 0.75

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['What a wonderful day!'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

What a wonderful day!‘‘ci”7‘".QI6Ei-pPal£8o8xVL£iQPCfaUYS.EB-:g“(aMBo4!I,”j( dBDCRtI”“kF8
V7&“!R'1’en'AkdhH! 5eh s5Kgcex;n
sV)“dgJ2Hv;-3,)XrayqDgP")?1KOQmbE2.”
RLad'GLodDCCGpfV:Q'&8B‘0‘Vq”TXoBe(s!DQ Ie’H,WU.R0r”3.
p!v1,  ”xNA’vh:n&tjZ0CjN'N;”fKmAu;WXCl7KCHe’LgrLQ)?”8f-- )KmbIVvM3Q8EOnb8Gj7RGtgs)YA Z:,;qySs'ayZyp0U7E,'P6';0'£p‘£"h5gcSQMYF‘jNf66qx&zRDVD.yMjTh‘iaC8uFR-Ok0bKBje!i0f ,7),7dHe”o''Tueawa1 4NWTeiemI4ZOAYbGn7S£)VC.O(a "nUpdPV£R8&f-9k!Z2”aEg!VoKO p'-ed.&Klu"F?T8CreTaQ8XwBb6wg&,!8tl-XUu2,'z!p‘Zlo5w“Jhpei4Bon&YgFl”u?rkYDR3sNeo()“L37G2yl56‘cdav"Eo!f4ub--CK?n(ofse-R£ c0XwJE“2h(gyGT8&ZHfkmXnh!aGhxJf'nD9?"1RanG?yKPIsdFk3B9Bkp0Yz7)TU:£-89W"”
,5muncS&-'m)hOCWi"’W!w“W0t”o1;'TBADi.0l4TjM,tahMmZbb5O!s£QYtLF"‘4SPE“ap"16P5ISXCWZKMm?HUI)M4BnK
”(Ey!iki,WvGNZQ4yc!mPHK!my&dkv pN4o qEi£w-&TeX.k7htRz'8s9XVcfYEt1
YPN,XhtK3&”cQcaGqMiV”3“qcnnRA6bF;mdvT(q‘mLpgdSRGq sWikp(Ov)kxufbmAm-i9Vl0 75z6oy52asIW9 SXvfNHrecsp1zJWX?Ev£f IRp“JIV£rRrS:j3-M6”,,BOdN'BbJu0E2”O0mewRM-jN1Kov(NQae7T u?(’EtuJi2KcT8(
jHE?dVQ

Shrek|| seq_length = 500, BATCH_SIZE = 50, GRU(rnn_units, return_sequences=True, return_state=True, activation = "tanh", recurrent_activation='sigmoid', recurrent_activation='sigmoid'), Dense(vocab_size), epochs=35, temperature = 0.75

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['SHREK'])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

SHREKDX4(7OBNYEINEU1NENNKNYON71RHNAMYHV7VK/N1(((ABIONAYWPDEXA1                            Donkey, wet o gowe hack of.

                                          SHREK
                          Uh, hingt, I's seren mighsto.
 
                                                 DONKEY
                             Wome.-Ind awr wet and may it apbuspes there mrore 
                                      the stall the gures walle the wat and 
                     lona to na readstint the thest.

                                                        Oh, I's jugt aplay s ara. Sorre yin ta 
                     the geserd. nson thet rempyen 
                         a anlas biof the that and o and youn 
                                                  fuctit donte cereadlesin thereas beang at and 
              
                           Shreke dot's wanking on the and tet hes arout the rond 
                                      I god damry..
 
                                                

## VI. Next Steps

This is a very simple model with one GRU layer and then an output layer. However, considering how simple it is and the fact that we are predicting outputs character by character, the text it produces is pretty amazing. Though it still has a long way to go before publication.

There are many other RNN architectures you could try, such as adding additional hidden dense layers, replacing GRU with one or more LSTM layers, combining GRU and LSTM, etc...

You could also experiment with better text cleanup to make sure odd punctuation doesn't appear, or finding longer texts to use. If you combine texts from two authors, what happens? Can you generate a Jane Austen stageplay by combining austen and shakespeare texts?

Finally, there are a number of hyperparameters to tweak, such as temperature, epochs, batch size, sequence length, etc...